In [2]:
from workflow.scripts.utils import read_list_input_paths
import xarray as xr
from pyclim_noresm.general_util_funcs import global_avg
import yaml

In [9]:
dep, dname = read_list_input_paths(snakemake.input.paths_emissions)
burden, bname = read_list_input_paths(snakemake.input.paths, 2)
area_cello = xr.open_dataset(snakemake.input.path_area)
vname = snakemake.wildcards.variable

In [4]:
annual_emis = {}
annual_burd = {}
for m, ds in dep.items():
    tds = ds.copy().isel(time=slice(3,None))
    tds[dname] = ds[dname]*area_cello['cell_area']*1e-9
    annual_emis[m] = tds[dname].sum(dim=['lon','lat'])
for m, ds in burden.items():
    tds = ds.copy().isel(time=slice(3,None))
    tds[bname] = ds[bname]*area_cello['cell_area']*1e-9
    annual_burd[m] = tds[bname].sum(dim=['lon','lat'])


In [13]:
lftime = {}
lftime[vname] = {}
for m in annual_burd:
    lftime[vname]['data'] = [float(v) for v in (annual_burd[m]/(annual_emis[m]/365))]
    lftime[vname]['units'] = 'Days'

lftime['name'] = m

In [9]:
with open(snakemake.output.table,'w') as f:
    yaml.safe_dump(lftime, f,default_flow_style=False)